In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os, gc
import sys

from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

from catboost import CatBoostRegressor

from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lightgbm as lgb
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import seaborn as sns

train = pd.read_csv('train_P0.csv')

In [37]:
def mse(x_hat, x):
    # |x_hat| = (batch_size, dim)
    # |x| = (batch_size, dim)
    y = ((x - x_hat)**2).mean()
    
    return y

loss = mse

In [26]:
col_list = (list(train.columns))

In [27]:
print(col_list)

['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ID', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN', 'ATA_LT', 'DUBAI', 'BRENT', 'WTI', 'BDI_ADJ', 'PORT_SIZE', 'CI_HOUR', 'DENSITY', 'B', 'YEAR', 'MONTH', 'HOUR']


# U_WIND 결측치 catboost로 채우기

채워야할 칼럼들:
1. U_WIND
2. V_WIND
3. AIR_TEMPERATURE
4. BN
5. B

채우는 방법:  
1~4:  
나머지 결측치들 모두 평균으로 채우고, 해당 결측치들이 포함된 row는 test data로 두고, 나머지 row를 train data로 사용하여 catboost 적용  
5:  
채운 BN을 활용해서 공식 사용

In [28]:
train1 = train[:]

In [29]:
train1['V_WIND'].fillna(train1['V_WIND'].median(), inplace=True)
train1['AIR_TEMPERATURE'].fillna(train1['AIR_TEMPERATURE'].median(), inplace=True)
train1['BN'].fillna(train1['BN'].median(), inplace=True)
train1['B'].fillna(train1['B'].median(), inplace=True)

C:\Users\kjsjh\AppData\Local\Temp\ipykernel_44576\3445198505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['V_WIND'].fillna(train1['V_WIND'].median(), inplace=True)
C:\Users\kjsjh\AppData\Local\Temp\ipykernel_44576\3445198505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1['AIR_TEMPERATURE'].fillna(train1['AIR_TEMPERATURE'].median(), inplace=True)
C:\Users\kjsjh\AppData\Local\Temp\ipykernel_44576\3445198505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [30]:
train1.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                     0
AIR_TEMPERATURE            0
BN                         0
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
DENSITY                    0
B                          0
YEAR                       0
MONTH                      0
HOUR                       0
dtype: int64

In [31]:
test1 = train1[train1.U_WIND.isna()]
train1 = train1[~train1.U_WIND.isna()]

In [32]:
col_list_ = []
for c in col_list:
    if c == 'U_WIND':
        continue
    col_list_.append(c)
    
train_x = train1[col_list_]
train_y = train1['U_WIND']

test_x = test1[col_list_]

In [33]:
# object형 변수는 cat_features에 추가
cat_features = [f for f in train_x.columns if train_x[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(train_x, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
[ 0  1  2  4 12 13]


In [34]:
test_x.shape[0]

153486

In [ ]:
cat_models={}

def cat_kfold(max_depth, learning_rate, random_seed):
    
    folds=StratifiedKFold(n_splits=10, shuffle=True, random_state=55)
    outcomes=[]
    sub=np.zeros((test_x.shape[0], 1))  
    
    for seed in random_seed:
        for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
            print(f'===================================={n_fold+1}============================================')
            
            X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
            y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

            # early_stopping 50에서 가장 좋은 점수를 내는 learning_rate를 활용
            cat = CatBoostRegressor(n_estimators=3000, max_depth=max_depth, random_seed=seed, learning_rate=learning_rate, bootstrap_type ='Bernoulli')
            cat.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_val, y_val)],
                  early_stopping_rounds=50, cat_features=cat_features,
                  verbose=100)

            cat_models[n_fold] = cat

            # val 데이터 예측
            predictions = cat.predict_proba(X_val)
            # test 데이터 예측
            test_predictions = cat.predict_proba(test_x)

            # val 데이터 예측 logloss 값 저장
            logloss=log_loss(to_categorical(y_val), predictions)
            outcomes.append(logloss)
            print(f"FOLD {n_fold+1} : logloss:{logloss}")

            # test 데이터 예측 결과 종합
            # 최종 적으로는 kolds 횟수 만큼 나눠서 평균 값을 활용
            sub+=test_predictions

            print(f'================================================================================\n\n')

    # 저장된 val 데이터 예측 logloss 값의 평균 값으로 성능을 비교
    mean_outcome=np.mean(outcomes)
    print("Mean:{}".format(mean_outcome))
    
    return sub/(folds.n_splits * len(random_seed))